# Devices
This tutorial is aimed at explaining the entirety of writing Device drivers to interface with lab equipment. Let's take a look at the most basic Device, which is found in the Core tutorial:

In [ ]:
from emergent.core import Device, Knob

class TestDevice(Device):
    ''' Device driver for the virtual network in the 'basic' example. '''
    X = Knob('X')
        
    
dev = TestDevice('dev', hub=None, params={})

We've defined a single knob called X. We can interact with it like this:

In [ ]:
dev.X = 2
print(dev.X)

## Device commands
So far, the knob is only a virtual object. In order to implement device control, we override the default setter method with whatever device command you want to send:

In [4]:
from emergent.core import Device, Knob

class TestDevice(Device):
    ''' Device driver for the virtual network in the 'basic' example. '''
    X = Knob('X')

#     @X.setter
#     def X(self, x):
#         print('Sending device command to update X to %f'%x)
#     def __init__(self, name, hub, params):
#         self.knobs = []
#         return
    
dev = TestDevice('dev', hub=None, params={})
dev2 = TestDevice('dev', hub=None, params={})
dev.X=3
dev.X=4
# dev.X

# print(dev.X, dev2.X)


Add boilerplate to X
Adding boilerplate


AttributeError: 'TestDevice' object has no attribute '__X'

In [3]:
dev.X

Adding boilerplate


4

Notice that the new setter method we defined doesn't explicitly redefine the variable - this is handled behind the scenes in the Device.\_add_boilerplate() method. You only need to define the device-specific code that you want to execute, and EMERGENT automatically adds the boilerplate to keep track of the virtual state.

## Device queries
The default Knob behavior is to store the last value defined by the user. This can lead to desynchronization between the virtual and physical values if someone turns a real knob in the lab. This can be remedied by overriding the getter method to request the real state from the device. This is not yet implemented due to a few technical reasons: overriding the getter requires replacing the entire Knob (defining a setter as well), and this breaks the auto-boilerplate-generator.

# A word on properties
The original version of EMERGENT represented device states through dictionaries. This representation is still used to communicate between Devices and Hubs; for example, you can access the state like this:

In [ ]:
dev._state()

You can also set the state using the actuate() method:

In [ ]:
dev.actuate({'X':5})

The new property-based state representation has several advantages over the dict-based representation:
* Quality of life: setting the property directly is quicker to type than passing a dict into the actuate method. Knob declaration is also simpler and cleaner with this method.
* Better memory compartmentalization: updating individual properties instead of a state dict reduces memory overlap between knobs, which will be useful in future parallelization efforts. Note that memory is still shared by the Hub.
* Device synchronization: the previous dict-based method simply logged the last state sent to the device, which could become unsynchronized with the device if a physical knob was turned in the lab. Now, you can override the getter method to request the actual state from the device.

# Properties of instances

In [ ]:
def Knob(name):
    def getter(self):
        print('Getting')
        _name = '_%s'%name
        if _name in self.__dict__:
            return self.__dict__[_name]
        
    def setter(self, newval):
        print('Setting')
        _name = '_%s'%name

        self.__dict__[_name] = newval
    return property(getter, setter)

        
from emergent.core import Device

class TestDevice(Device):
    ''' Device driver for the virtual network in the 'basic' example. '''
    X = Knob('X')
    
    
dev = TestDevice('dev', hub=None, params={})
dev2 = TestDevice('dev2', hub=None, params={})

dev.X = 1
dev2.X = 2

print(dev.X, dev2.X)

In [ ]:
dev2.__dict__

In [ ]:
class Foo:
    @property
    def x(self):
        if '_x' in self.__dict__:
            return self.__dict__['_x']
    
    @x.setter
    def x(self, newval):
        self.__dict__['_x'] = newval
a = Foo()
b = Foo()
a.x=1
b.x = 2


print(a.x, b.x)

In [ ]:
a.x